In [182]:
import numpy as np
import pandas as pd

#housing = pd.read_csv('https://raw.githubusercontent.com/ageron/handson-ml/master/datasets/housing/housing.csv')
#housing.head(10)

housing = pd.read_csv('housing.csv')
housing.head(10)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY
5,-122.25,37.85,52.0,919.0,213.0,413.0,193.0,4.0368,269700.0,NEAR BAY
6,-122.25,37.84,52.0,2535.0,489.0,1094.0,514.0,3.6591,299200.0,NEAR BAY
7,-122.25,37.84,52.0,3104.0,687.0,1157.0,647.0,3.1200,241400.0,NEAR BAY
8,-122.26,37.84,42.0,2555.0,665.0,1206.0,595.0,2.0804,226700.0,NEAR BAY
9,-122.25,37.84,52.0,3549.0,707.0,1551.0,714.0,3.6912,261100.0,NEAR BAY


In [183]:
median = housing['total_bedrooms'].median()
housing['total_bedrooms'] = housing['total_bedrooms'].fillna(median)

In [184]:
housing['rooms_per_household'] = housing['total_rooms'] / housing['households']
housing['bedrooms_per_room'] = housing['total_bedrooms'] / housing['total_rooms']
del housing['total_bedrooms'], housing['total_rooms']

In [185]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

col_list = list(housing)
col_list.remove('ocean_proximity')
col_list.remove('median_house_value')

housing_numeric = housing[col_list]
housing_scaled = scaler.fit_transform(housing_numeric)

housing_scaled_df = pd.DataFrame(housing_scaled, index=housing_numeric.index, columns=housing_numeric.columns)

housing = pd.concat([housing_scaled_df, housing['median_house_value'], housing['ocean_proximity']], axis=1)
housing.head()

,longitude,latitude,housing_median_age,population,households,median_income,rooms_per_household,bedrooms_per_room,median_house_value,ocean_proximity
0,-1.327835,1.052548,0.982143,-0.974429,-0.977033,2.344766,0.628559,-1.029988,452600.0,NEAR BAY
1,-1.322844,1.043185,-0.607019,0.861439,1.669961,2.332238,0.327041,-0.888897,358500.0,NEAR BAY
2,-1.332827,1.038503,1.856182,-0.820777,-0.843637,1.782699,1.155620,-1.291686,352100.0,NEAR BAY
3,-1.337818,1.038503,1.856182,-0.766028,-0.733781,0.932968,0.156966,-0.449613,341300.0,NEAR BAY
4,-1.337818,1.038503,1.856182,-0.759847,-0.629157,-0.012881,0.344711,-0.639087,342200.0,NEAR BAY


In [186]:
housing = pd.get_dummies(housing)
housing.tail(10)

,longitude,latitude,housing_median_age,population,households,median_income,rooms_per_household,bedrooms_per_room,median_house_value,ocean_proximity_<1H OCEAN,ocean_proximity_INLAND,ocean_proximity_ISLAND,ocean_proximity_NEAR BAY,ocean_proximity_NEAR OCEAN
20630,-0.873626,1.712691,-1.401600,-0.148774,-0.142654,-0.159688,0.203541,-0.344941,112000.0,0,1,0,0,0
20631,-0.913557,1.731419,-1.083767,-0.199108,-0.176657,-0.185691,0.289734,-0.430777,107200.0,0,1,0,0,0
20632,-0.938513,1.698646,-1.083767,-0.334215,-0.299591,-0.392505,0.240238,-0.527329,115600.0,0,1,0,0,0
20633,-0.978444,1.665873,-0.130270,-0.303308,-0.307438,-0.695435,0.006478,-0.240889,98300.0,0,1,0,0,0
20634,-0.993418,1.703328,-0.050812,-0.339513,-0.406831,-0.083258,0.545678,-0.680673,116800.0,0,1,0,0,0
20635,-0.758826,1.801647,-0.289187,-0.512592,-0.443449,-1.216128,-0.155023,0.165994,78100.0,0,1,0,0,0
20636,-0.818722,1.806329,-0.845393,-0.944405,-1.008420,-0.691593,0.276881,0.021671,77100.0,0,1,0,0,0
20637,-0.823713,1.778237,-0.924851,-0.369537,-0.174042,-1.142593,-0.090318,0.021134,92300.0,0,1,0,0,0
20638,-0.873626,1.778237,-0.845393,-0.604429,-0.393753,-1.054583,-0.040211,0.093467,84700.0,0,1,0,0,0
20639,-0.833696,1.750146,-1.004309,-0.033977,0.079672,-0.780129,-0.070443,0.113275,89400.0,0,1,0,0,0


In [187]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(housing, test_size=0.2, random_state=45)
print("# of train set: %.0f. # of test set: %.0f" % (train_set.shape[0], test_set.shape[0]))

# of train set: 16512. # of test set: 4128


In [188]:
train_set_features = train_set.drop('median_house_value', axis=1)
train_set_target = train_set['median_house_value'].copy()

In [189]:
from sklearn.linear_model import  LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

lin_reg = LinearRegression()
lin_reg.fit(train_set_features, train_set_target)
print("Learned Parameters : ")
print(lin_reg.coef_)
print(lin_reg.intercept_)

print("Train_r2_score : ", lin_reg.score(train_set_features, train_set_target))

Learned Parameters : 
[-55774.14086907 -56964.69903623  12971.17103303 -53562.80929239
  57272.02939275  74147.51210095   6564.46012412  10459.02523705
 -25921.29448802 -63911.36887214 143057.05886283 -30846.20870803
 -22378.18679464]
244824.9222886056
Train_r2_score :  0.6534060592966192


In [190]:
test_set_features = test_set.drop('median_house_value', axis=1)
test_set_target = test_set['median_house_value'].copy()

final_model = lin_reg
final_predictions = final_model.predict(test_set_features)

print("Test_r2_score : ", r2_score(test_set_target, final_predictions))

delta = test_set_target - final_predictions
print("Errors (%) in housing value prediction : ", np.mean(np.abs(delta)/test_set_target))

final_mse = mean_squared_error(test_set_target, final_predictions)
final_rmse = np.sqrt(final_mse)
print("RMSE is : ", final_rmse)

Test_r2_score :  0.6308608545588803
Errors (%) in housing value prediction :  0.2861199789930685
RMSE is :  69987.25423203211


In [191]:
print(test_set_features.iloc[2])
print(test_set_features.iloc[43])

x1 = np.array(test_set_features.iloc[2])
x2 = np.array(test_set_features.iloc[43])
prediction1 = np.dot(lin_reg.coef_, x1) + lin_reg.intercept_
prediction2 = np.dot(lin_reg.coef_, x2) + lin_reg.intercept_

print("2nd sample-predicted {}.  Real value: {}".format(prediction1, test_set_target.iloc[2]))
print("43rd sample-predicted {}.  Real value: {}".format(prediction2, test_set_target.iloc[43]))

longitude                     0.813436
latitude                     -0.792107
housing_median_age            0.108104
population                   -0.512592
households                   -0.498377
median_income                 0.336318
rooms_per_household          -0.106685
bedrooms_per_room            -0.328572
ocean_proximity_<1H OCEAN     1.000000
ocean_proximity_INLAND        0.000000
ocean_proximity_ISLAND        0.000000
ocean_proximity_NEAR BAY      0.000000
ocean_proximity_NEAR OCEAN    0.000000
Name: 10112, dtype: float64
longitude                    -0.728878
latitude                      1.642463
housing_median_age           -1.481058
population                   -0.132879
households                    0.029976
median_income                 0.108870
rooms_per_household           0.287740
bedrooms_per_room            -0.717413
ocean_proximity_<1H OCEAN     0.000000
ocean_proximity_INLAND        1.000000
ocean_proximity_ISLAND        0.000000
ocean_proximity_NEAR BAY      0.0000

In [192]:
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

model_ridge = Ridge(alpha=100)
model_ridge.fit(train_set_features, train_set_target)

print("Train_score : ", model_ridge.score(train_set_features, train_set_target))

ridge_predicted = model_ridge.predict(test_set_features)
print("Test_score : ", r2_score(test_set_target, ridge_predicted))

delta = test_set_target - ridge_predicted
print("Errors in housing value prediction : ", np.mean(np.abs(delta)/test_set_target))

final_ridge_mse = mean_squared_error(test_set_target, ridge_predicted)
final_ridge_rmse = np.sqrt(final_ridge_mse)

print("final_ridge_RMSE : ", final_ridge_rmse)

import warnings
warnings.filterwarnings(action='ignore')

Train_score :  0.6525115810044481
Test_score :  0.6333956237676854
Errors in housing value prediction :  0.28415587884460797
final_ridge_RMSE :  69746.54943237505


In [193]:
from sklearn.linear_model import Lasso

model_lasso = Lasso(alpha=1000)
model_lasso.fit(train_set_features, train_set_target)

print("Train_score : ", model_lasso.score(train_set_features, train_set_target))

lasso_predicted = model_lasso.predict(test_set_features)
print("Test_score : ", r2_score(test_set_target, lasso_predicted))

delta = test_set_target - lasso_predicted
print("Errors in housing value prediction : ", np.mean(np.abs(delta)/test_set_target))

final_lasso_mse = mean_squared_error(test_set_target, lasso_predicted)
final_lasso_rmse = np.sqrt(final_lasso_mse)
print("RMSE is : ", final_lasso_rmse)

Train_score :  0.6477400839070926
Test_score :  0.6353348735073737
Errors in housing value prediction :  0.28205827189150406
RMSE is :  69561.83352439076


In [194]:
from sklearn.neural_network import MLPRegressor

model_MLP = MLPRegressor(activation = 'relu', hidden_layer_sizes = (32,64,32), max_iter = 1000, solver = 'lbfgs', random_state = 0)
model_MLP.fit(train_set_features, train_set_target)

print("Train_score : ", model_MLP.score(train_set_features, train_set_target))

MLP_predicted = model_MLP.predict(test_set_features)
print("Test_score : ", r2_score(test_set_target, MLP_predicted))

delta = test_set_target - MLP_predicted
print("Errors in housing value prediction : ", np.mean(np.abs(delta)/test_set_target))

final_MLP_mse = mean_squared_error(test_set_target, MLP_predicted)
final_MLP_rmse = np.sqrt(final_MLP_mse)
print("RMSE is : ", final_MLP_rmse)

Train_score :  0.8472736143958429
Test_score :  0.8103952921570186
Errors in housing value prediction :  0.18200279199223732
RMSE is :  50158.95298421304
